In [ ]:
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
class Game:
    def __init__(self,board =[[5,5,5,5,5],[5,5,5,5,5]],turn = 1, score1 = 0,score2 = 0):
        self.finish = False
        r=0
        self.board= [[5,5,5,5,5], [5,5,5,5,5]]
        for i in range(2):
            for j in range(5):
                self.board[i][j]=board[i][j]
        self.score1 = score1
        self.score2 = score2
        self.turn = turn
    def put(self,board,turn,score1,score2):
        self.board= [[5,5,5,5,5], [5,5,5,5,5]]
        for i in range(2):
            for j in range(5):
                self.board[i][j]=board[i][j]
        self.score1 = score1
        self.score2 = score2
        self.turn = turn
    def do_move(self,move):
        i = 0
        last = 0
        MOVE = 0
        if(move == 6):
            MOVE = 10 + self.turn
            move = 3
        balls = self.board[self.turn-1][move-1]
        if(balls <= 1):
            return -1
        if(MOVE <= 10):
            MOVE = move - 1 + 5 *(self.turn - 1)
        self.board[self.turn -1][move-1] = 0
        if(MOVE in [0,1,2,9,8,7]):
            if MOVE == 0: i=0
            elif MOVE == 9: i=5
            elif MOVE in [1,2]: i=10-MOVE
            else: i=MOVE-4
            for b in range(balls):
                k=i%10
                if(k<5):
                    self.board[1][k] += 1
                    if b==balls-1: last = self.board[1][k]
                    if(last in [2,4] and self.turn==1):
                        self.board[1][k]=0
                        self.score1 += last
                        for l in range(1,k+1):
                            last = self.board[1][k-l]
                            if(last in [2,4]):
                                self.score1 += last
                                self.board[1][k-l]=0
                            else: return 1
                        return 1
                else:
                    self.board[0][9-k] += 1
                    if b==balls-1: last = self.board[0][9-k]
                    if(last in [2,4] and self.turn==2):
                        self.board[0][9-k]=0
                        self.score2 += last
                        for l in range(1,k-4):
                            last = self.board[0][9-k+l]
                            if(last in [2,4]):
                                self.score2 += last
                                self.board[0][9-k+l]=0
                            else: return 1
                        return 1
                i+=1
        else:
            if MOVE == 4: i = 0
            elif MOVE == 5: i = 5
            elif MOVE == 6: i = 4
            elif MOVE == 3: i = 9
            elif MOVE == 11:i = 8
            else: i = 3
            for b in range(balls):
                k=i%10
                if(k<5):
                    self.board[1][4-k] += 1
                    if b==balls-1: last = self.board[1][4-k]
                    if(last in [2,4] and self.turn==1):
                        self.board[1][4-k]=0
                        self.score1 += last
                        for l in range(1,k+1):
                            last = self.board[1][4-k+l]
                            if(last in [2,4]):
                                self.score1 += last
                                self.board[1][4-k+l]=0
                            else: return 1
                        return 1
                else:
                    self.board[0][k-5] += 1
                    if b==balls-1: last = self.board[0][k-5]
                    if(last in [2,4] and self.turn==2):
                        self.board[0][k-5]=0
                        self.score2 += last
                        for l in range(1,k-4):
                            last = self.board[0][k-5-l]
                            if(last in [2,4]):
                                self.score2 += last
                                self.board[0][k-5-l]=0
                            else: return 1
                        return 1
                i+=1
        return 0    
    def check_turn(self):
        for j in range(5):
            if(self.board[self.turn-1][j] >= 2):
                return False
        return True
    def check_finish(self):
        for i in range(2):
            for j in range(5):
                if(self.board[i][j] >= 2):
                    return False
        self.score1 += sum(self.board[0])
        self.score2 += sum(self.board[1])
        self.finish = True
        return True
    def print_game(self,sc =True):
        for i in range(2):
            print(self.board[i][0],self.board[i][1],self.board[i][2],self.board[i][3],self.board[i][4])
        print("[ ",self.score1," , ",self.score2," ]")
    def flip_turn(self):
        if self.turn==1: self.turn=2
        else: self.turn=1
            
class Player:
    def __init__(self,PType="human",agent = None,depth=3):
        self.type = PType
        self.depth = depth
        if(PType=="DQN"):
            if(agent != None): self.agent = agent
            else: self.agent = DQNAgent()
    def get_move(self,game):
        if(self.type == "human"):
            return int(input("Enter move:"))
        elif(self.type == "random"):
            return np.random.randint(1,6)
        elif(self.type=="DQN"):
            return np.argmax(self.agent.model.predict(np.array(game.board).reshape((1,10)))[0])
        elif(self.type=="minmax"):
            return min_max(game.board,self.depth,game.turn,game.score1,game.score2)
        else:
            return 0
def min_max(board,depth,turn,s1,s2):
    G = Game(board=board,turn=turn,score1=s1,score2=s2)
    Max = -100
    Best = np.random.randint(1,6)
    for i in range(1,7):
        G.put(board,turn,s1,s2)
        if(G.do_move(i)==-1):
            continue
        d = r_min_max(G.board,G.turn,G.score1,G.score2,depth,turn)
        if(d > Max):
            Max = d
            Best = i
    del(G)
    return Best

def r_min_max(board,turn,s1,s2,depth,orig):
    depth -= 1
    if depth==0: 
        if orig==2: return s2-s1
        return s1-s2
    G= Game(board,turn,s1,s2)
    G.flip_turn()
    turn=G.turn
    is_max = False
    MinMax = 100
    if G.turn==orig: 
        is_max=True
        MinMax = -100
    if(G.check_finish()):
        if orig==1: return G.score1-G.score2
        return G.score2-G.score1
    elif(G.check_turn()):
        if orig==1: return G.score1-G.score2
        return G.score2-G.score1
    for i in range(1,7):
        G.put(board,turn,s1,s2)
        if G.do_move(i)==-1:
            delt = G.score1-G.score2
            if orig==1: return delt
            return -1*delt
        d = r_min_max(G.board,G.turn,G.score1,G.score2,depth,orig)
        if(is_max and d>MinMax):
            MinMax = d
        elif(not is_max and d<MinMax):
            MinMax = d
    del(G)
    return MinMax
            
        
        
    return 0
def plot_seaborn(array_counter, array_score):
    sns.set(color_codes=True)
    ax = sns.regplot(np.array([array_counter])[0], np.array([array_score])[0], color="b", x_jitter=.1, line_kws={'color':'green'})
    ax.set(xlabel='games', ylabel='score')
    plt.show()
def timer(st=False,p=0):
    if st:
        return time.time()
    else:
        print("ok")
        print("Total Time = %f" % (p-time.time()))
        return p-time.time()
def run(player1,player2,numGames):
    win1 = 0
    win2 = 0
    tie = 0
    for gm in range(numGames):
        print(gm+1)
        g=Game()
        while(not g.check_finish()):
            r=-1
            while(r==-1 and not g.check_turn()):
                if g.turn==1: m = player1.get_move(g)
                else: m = player2.get_move(g)
                r = g.do_move(m)
                if(r==-1): g.do_move(np.random.randint(1,6))
            #print(g.turn,m)
            #g.print_game()
            g.flip_turn()
            if(player1.type=="human" or player2.type=="human"): print(m);g.print_game()
        #g.print_game()
        if(g.score1>g.score2): win1 += 1
        elif(g.score1<g.score2): win2 += 1
        else: tie += 1
        del(g)
    print("Win 1 =",win1,"Win 2 =",win2,"Tie =",tie)
    return 0

def train(agent = DQNAgent(),trainer = Player("random"),numGames = 100):
    print("Training on",numGames," games.")
    score_plot = []
    counter_plot = []
    for gm in range(numGames):
        #print("Playing game #",gm+1)
        g=Game(turn = 2)
        agent.epsilon = numGames/2 - gm
        while(not g.check_finish()):
            r = -1
            old_board = np.array(g.board)
            old_scores = [g.score1,g.score2]
            g.turn = 2
            
            #print("Agent playing")
            c1 = not g.finish
            while(r==-1 and not g.check_turn()):
                if random.randint(0, int(numGames/2)) < agent.epsilon:
                    m = random.randint(1,6)
                    final_move = keras.utils.to_categorical(m-1, num_classes=6)
                    r = g.do_move(m)
                    #print("random =",m)
                else:
                    m = np.argmax(agent.model.predict(np.array(g.board).reshape((1,10)))[0])+1
                    #print("prediction =",m)
                    final_move = keras.utils.to_categorical(m-1, num_classes=6)
                    r = g.do_move(m)
                    if(r==-1):
                        while(r==-1 and not g.check_turn()):
                            m = random.randint(1,6)
                            final_move = keras.utils.to_categorical(m-1, num_classes=6)
                            r = g.do_move(m)
                c1 = g.check_turn()
            r=-1
            c2= not g.finish
            g.turn = 1
            #print(m)
            #g.print_game()
            #print("Trainer playing")
            while(r==-1 and not g.check_turn()):
                m = trainer.get_move(g)
                #print(m,g.turn)
                r = g.do_move(m)
                if r ==-1: g.do_move(random.randint(1,6))
                c2 = g.check_turn()
            #print(m)
            #g.print_game()
            #print("final:",m)
            if(not c1):
                #print("saving a move")
                Estart=time.time()
                new_board = np.array(g.board)
                new_scores = [g.score1,g.score2]
                reward = agent.set_reward(old_scores[1]-old_scores[0],new_scores[1]-new_scores[0])
                agent.train_short_memory(old_board, final_move, reward, new_board, g.finish)
                agent.remember(old_board.reshape((10)), final_move, reward, new_board.reshape((10)),g.finish)
                #print("move saved with parameters:",old_board, final_move, reward, new_board, g.finish)
                #start=time.time()-start
                #eprint("getting a state time = %f" % (start))
        #start= time.time()
        agent.replay_new(agent.memory)
        #start=time.time()-start
        #print("replay new time = %f" % (start))
        print("Game:",gm+1,"Scores:",g.score1,g.score2,"Aquired data:",len(agent.memory))
        score_plot.append(g.score2-g.score1)
        counter_plot.append(gm)
        del(g)
        #input()
    print("Finished training!")
    agent.model.save_weights('Mungala_weights.hdf5')
    plot_seaborn(counter_plot, score_plot)
    return 0
#p2 = Player("DQN",agent=Agent,depth=20)
p1 = Player("human",depth=5)
p2 = Player("minmax",depth=5)
run(p1,p2,1000)
#Agent = DQNAgent()
#trainer = Player("minmax",depth=5)
#train(Agent,trainer,100)

W0830 19:04:24.801859 12060 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 19:04:25.336315 12060 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 19:04:25.435526 12060 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0830 19:04:25.536000 12060 deprecation_wrapper.py:119] From c:\users\yazeed\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default 

1


In [75]:
p=True
for state, action, reward, next_state, done in Agent.memory:
    target = reward
    if not done:
        target = reward + Agent.gamma * np.amax(Agent.model.predict(np.array([next_state]))[0])
    target_f = Agent.model.predict(np.array([state]))
    target_f[0][np.argmax(action)] = target
    if p:
        states=state
        targets = target_f
        p=False
    else:
        states=np.vstack((states, state))
        targets=np.vstack((targets,target_f))
#print(states.shape)
model
Agent.model.fit(states.reshape(len(states),10), targets.reshape(len(targets),6), epochs=10)

Epoch 1/10
4898/4898 [==============================] - 1s 287us/step - loss: 128.3188
Epoch 2/10
4898/4898 [==============================] - 1s 265us/step - loss: 128.3444
Epoch 3/10
4898/4898 [==============================] - 1s 255us/step - loss: 128.3083
Epoch 4/10
4898/4898 [==============================] - 1s 253us/step - loss: 128.2767
Epoch 5/10
4898/4898 [==============================] - 1s 250us/step - loss: 128.2850
Epoch 6/10
4898/4898 [==============================] - 1s 248us/step - loss: 128.2571
Epoch 7/10
4898/4898 [==============================] - 1s 244us/step - loss: 128.2163
Epoch 8/10
4898/4898 [==============================] - 1s 238us/step - loss: 128.1803
Epoch 9/10
4898/4898 [==============================] - 1s 230us/step - loss: 128.1555
Epoch 10/10
4898/4898 [==============================] - 1s 216us/step - loss: 128.1194 


In [71]:
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=10))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=6, activation='softmax'))
opt = Adam(Agent.learning_rate)
model.compile(loss='mse', optimizer=opt,metrics=['accuracy'])
model.fit(states.reshape(len(states),10), targets.reshape(len(targets),6), epochs=100)

Epoch 1/100
1598/1598 [==============================] - 2s 2ms/step - loss: 129.3975 - acc: 0.1471
Epoch 2/100
1598/1598 [==============================] - 0s 296us/step - loss: 129.3588 - acc: 0.1865
Epoch 3/100
1598/1598 [==============================] - 0s 268us/step - loss: 129.3787 - acc: 0.1915
Epoch 4/100
1598/1598 [==============================] - 0s 270us/step - loss: 129.3589 - acc: 0.2234
Epoch 5/100
1598/1598 [==============================] - 0s 295us/step - loss: 129.3625 - acc: 0.2616
Epoch 6/100
1598/1598 [==============================] - 0s 299us/step - loss: 129.2913 - acc: 0.2866
Epoch 7/100
1598/1598 [==============================] - 0s 300us/step - loss: 129.2551 - acc: 0.3204
Epoch 8/100
1598/1598 [==============================] - 0s 309us/step - loss: 129.2880 - acc: 0.3548
Epoch 9/100
1598/1598 [==============================] - 0s 295us/step - loss: 129.3006 - acc: 0.3892
Epoch 10/100
1598/1598 [==============================] - 0s 294us/step - loss: 129.

KeyboardInterrupt: 

In [2]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
import random
import numpy as np
import pandas as pd
from operator import add


class DQNAgent(object):

    def __init__(self):
        self.reward = 0
        self.gamma = 0.9
        self.dataframe = pd.DataFrame()
        self.short_memory = np.array([])
        self.agent_target = 1
        self.agent_predict = 0
        self.learning_rate = 0.0005
        self.model = self.network()
        #self.model = self.network("weights.hdf5")
        self.epsilon = 0
        self.actual = []
        self.memory = []
    def get_state(self, game):
        return np.asarray(game.board)
    def set_reward(self, old_delta, new_delta):
        self.reward = 0
        self.reward = (new_delta - old_delta) * 10
        return self.reward

    def network(self, weights=None):
        model = Sequential()
        model.add(Dense(units=120, activation='relu', input_dim=10))
        model.add(Dropout(0.15))
        model.add(Dense(units=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(units=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(units=6, activation='softmax'))
        opt = Adam(self.learning_rate)
        model.compile(loss='mse', optimizer=opt)

        if weights:
            model.load_weights(weights)
        return model

    def remember(self, state, action, reward, next_state,done):
        self.memory.append((state, action, reward, next_state,done))

    def replay_new(self, memory):
        if len(memory) > 1000:
            minibatch = random.sample(memory, 1000)
        else:
            minibatch = memory
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
            target_f = self.model.predict(np.array([state]))
            target_f[0][np.argmax(action)] = target
            self.model.fit(np.array([state]), target_f, epochs=1, verbose=0)
        

    def train_short_memory(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = reward + self.gamma * np.amax(self.model.predict(next_state.reshape((1, 10)))[0])
        target_f = self.model.predict(state.reshape((1, 10)))
        target_f[0][np.argmax(action)] = target
        self.model.fit(state.reshape((1, 10)), target_f, epochs=1, verbose=0)